In [85]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models import Word2Vec
import collections
from collections import Counter
import PyPDF2
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher

import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

from sklearn.feature_extraction.text import CountVectorizer

import os
import collections
from os import listdir
from os.path import isfile, join
from io import StringIO

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import svm
import xgboost
from sklearn import datasets, ensemble

## Requirement document Bag of words
# ============================================================

In [86]:
# This section is resposible for extracting the requirement document
def reqDocExtraction(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()

alltext=[]
def extract_text(pdf_path):
    for page in reqDocExtraction(pdf_path):
        alltext.append(page)

extract_text('./dataset/Job description.pdf')

# you may also want to remove whitespace characters like `\n` at the end of each line
reqDocExtractedData = [x.strip() for x in alltext]
print(reqDocExtractedData)


["Machine Learning Engineering13585ABCKnowledge and InnovationWhat you’ll doYou will focus on researching, building, and designingself-running artificial intelligence (AI)systems to automate predictive models. You are responsibleto design and create the AIalgorithms capable of learning and making predictionsthat define machine learning (ML).Experience and qualifications●Bachelor's or Master's degree (mention the courseas per requirement)●0-2 years of work experience providing analytics solutionsin a commercial settingTechnical expertiseMust have●Machine Learning, Clustering, Logistic Regression,Classification●Different libraries such as SciKit Learn, NumPy, Pandas,Matplotlib, Seaborn●Deep learning frameworks such as Tensorflow, Keras,PyTorch and application of NeuralNetworks and models. CNN, RNN, GANs●Familiar with Natural Language Processing and associatedlibraries like NLTK, SpaCy,Beautiful Soup●PySpark, Hadoop, and Big Data Pipelines●Data science methodology from exploratory data an

In [87]:
x=[]
for line in reqDocExtractedData:
    tokens=word_tokenize(line)
    tok=[w.lower() for w in tokens]
    table=str.maketrans('','',string.punctuation)
    strpp=[w.translate(table) for w in tok]
    words=[word for word in strpp if word.isalpha()]
    stop_words=set(stopwords.words('english'))
    words=[w for w in words if not w in stop_words]
    x.append(words)

print(len(x))
print(x)

2
[['machine', 'learning', 'innovationwhat', 'doyou', 'focus', 'researching', 'building', 'designingselfrunning', 'artificial', 'intelligence', 'ai', 'systems', 'automate', 'predictive', 'models', 'responsibleto', 'design', 'create', 'aialgorithms', 'capable', 'learning', 'making', 'predictionsthat', 'define', 'machine', 'learning', 'ml', 'experience', 'master', 'degree', 'mention', 'courseas', 'per', 'requirement', 'years', 'work', 'experience', 'providing', 'analytics', 'solutionsin', 'commercial', 'settingtechnical', 'expertisemust', 'learning', 'clustering', 'logistic', 'regression', 'libraries', 'scikit', 'learn', 'numpy', 'pandas', 'matplotlib', 'learning', 'frameworks', 'tensorflow', 'keras', 'pytorch', 'application', 'neuralnetworks', 'models', 'cnn', 'rnn', 'natural', 'language', 'processing', 'associatedlibraries', 'like', 'nltk', 'spacy', 'beautiful', 'hadoop', 'big', 'data', 'science', 'methodology', 'exploratory', 'data', 'analysis', 'feature', 'engineering', 'modelselecti

In [88]:
def combineTwoList(s):
    str1 = " "  
    return (str1.join(s))

str1 = combineTwoList(x[0])
str2 = combineTwoList(x[1])
finalStr = str1 + " " + str2
finalData = [finalStr]
print(finalData)

['machine learning innovationwhat doyou focus researching building designingselfrunning artificial intelligence ai systems automate predictive models responsibleto design create aialgorithms capable learning making predictionsthat define machine learning ml experience master degree mention courseas per requirement years work experience providing analytics solutionsin commercial settingtechnical expertisemust learning clustering logistic regression libraries scikit learn numpy pandas matplotlib learning frameworks tensorflow keras pytorch application neuralnetworks models cnn rnn natural language processing associatedlibraries like nltk spacy beautiful hadoop big data science methodology exploratory data analysis feature engineering modelselection deployment model scale nlp architectures computer vision modelsin productionconsidered advance techniques computer vision models object detection ocr tools reduce medium fordata transfers frameworks like django databases like mongodb nosql fir

In [89]:
#we will be creating count vector for the requirement document
CountVector=CountVectorizer()
bagOfWords=CountVector.fit_transform(finalData)
countVectDF = pd.DataFrame(bagOfWords.toarray(),columns=CountVector.get_feature_names()).head(10)

# countVectDF['Candidate_No'] = "Job Description"
# countVectDF = countVectDF.DataFrame(columns=countVectDF.columns)

countVectDF

,advance,ai,aialgorithms,analysis,analytics,application,architectures,artificial,associatedlibraries,automate,...,systems,techniques,tensorflow,time,tools,transfers,typefull,vision,work,years
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1,1


In [90]:
countTestVectDF = pd.DataFrame(columns=countVectDF.columns)
countTestVectDF

,advance,ai,aialgorithms,analysis,analytics,application,architectures,artificial,associatedlibraries,automate,...,systems,techniques,tensorflow,time,tools,transfers,typefull,vision,work,years


**MOTS:** We have some 106 words as our bags of words.

## Candidate one by one and creating their Count vector
# ====================================================================

In [91]:
#we are extracting one person profile
def candidateProfExtraction(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()

alltext=[]
def extract_text(pdf_path):
    for page in candidateProfExtraction(pdf_path):
        alltext.append(page)
        return alltext;

In [92]:
# Taking file patha nd open file one by one
mypath=r'./dataset/trainResumes'
allTrainingFiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
len(allTrainingFiles)

print(allTrainingFiles[1])

./dataset/trainResumes\candidate_001.pdf


In [93]:
currentCandFile=0;
while currentCandFile < len(allTrainingFiles):
    currentFile=allTrainingFiles[currentCandFile]
    candidateExtData = extract_text(currentFile)
    test_vector = CountVector.transform(candidateExtData)
    data = test_vector.toarray()
    currentVect=[]
    for i in range(106):
        currentVect.append(data[currentCandFile][i])
    countVectDF.loc[len(countVectDF.index)] = currentVect
    currentCandFile+=1;

countVectDF

,advance,ai,aialgorithms,analysis,analytics,application,architectures,artificial,associatedlibraries,automate,...,systems,techniques,tensorflow,time,tools,transfers,typefull,vision,work,years
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
88,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
89,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [94]:
countVectDF.to_excel("trainVectordata.xlsx")

## Data for final training for that we need to make the countVectDF with candidate name name and target columns
# =====================================================

In [95]:
# Reading the train file which contain all the candidate percentage
FirstDS = pd.read_csv('./dataset/train.csv')

countVectDF["Candidate_ID"] = FirstDS['CandidateID']
countVectDF["MatchPercentage"] = FirstDS['Match Percentage']
# countVectDF

# An SNS plot to see heatmap
# plt.figure(figsize=(25,13))
# sns.heatmap(countVectDF.corr(),annot=True,cmap="RdYlGn")
# plt.show()

In [96]:
# Creating One more dataset as equivalent as countVectDF
secondDS = countVectDF
secondDS.drop(columns='Candidate_ID',inplace=True)
X=secondDS.drop(columns='MatchPercentage')
y=secondDS['MatchPercentage']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [97]:
# Naive bayes algorithm
naiveBayes = MultinomialNB()
naiveBayes.fit(X_train,y_train)
pred=naiveBayes.predict(X_test)
NBayesAccuracy = accuracy_score(y_test,pred)
print(NBayesAccuracy)

#SVM algorithm
clf = svm.SVC()
clf.fit(X_train,y_train)
pred=clf.predict(X_test)
SVMAccuracy = accuracy_score(y_test,pred)
print(SVMAccuracy)

0.0
0.0


## Test Resumes now we need to make the prediction for the testData
# ==============================================================

In [99]:
#we are extracting one person profile

def candidateProfExtraction1(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            # close open handles
            converter.close()
            fake_file_handle.close()

alltext=[]
def extract_text1(pdf_path):
    for page in candidateProfExtraction1(pdf_path):
        alltext.append(page)
        return alltext;

In [100]:
# Taking file patha nd open file one by one
mypath=r'./dataset/testResumes'
allTestingFiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
len(allTestingFiles)

print(allTestingFiles[0])


./dataset/testResumes\candidate_004.pdf


In [101]:
currentCandFile1=0;
while currentCandFile1 < len(allTestingFiles):
    currentFile1=allTestingFiles[currentCandFile1]
    candidateExtData1 = extract_text1(currentFile1)
    test_vector1 = CountVector.transform(candidateExtData1)
    data1 = test_vector1.toarray()
    currentVect1=[]
    for j in range(106):
        currentVect1.append(data1[currentCandFile1][j])
    countTestVectDF.loc[len(countTestVectDF.index)] = currentVect1
    currentCandFile1+=1;

In [102]:
# countVectDF.shape()
countVectDF.head(10)

,advance,ai,aialgorithms,analysis,analytics,application,architectures,artificial,associatedlibraries,automate,...,techniques,tensorflow,time,tools,transfers,typefull,vision,work,years,MatchPercentage
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2,1,1,50
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,14
2,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,40
3,0,0,0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,49
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,37
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,45
6,0,0,0,3,2,0,0,0,0,0,...,0,0,0,1,0,0,0,2,0,38
7,0,0,0,3,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,57
8,0,0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,1,0,58
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,41


In [103]:
# countTestVectDF.shape()
countTestVectDF.head(10)

,advance,ai,aialgorithms,analysis,analytics,application,architectures,artificial,associatedlibraries,automate,...,systems,techniques,tensorflow,time,tools,transfers,typefull,vision,work,years
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,2,0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
9,0,0,0,1,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
countTestVectDF.to_excel("testVectordata.xlsx")  

In [ ]:
pred1=naiveBayes.predict(countTestVectDF)
# print(pred1)

pred = pd.DataFrame(pred1)
pred.shape


pred.to_excel('Testing output.xlsx')


In [ ]:

pred2=clf.predict(countTestVectDF)
# print(pred2)

pred = pd.DataFrame(pred2)
pred.shape


pred.to_excel('Testing output.xlsx')

In [115]:
params = {'n_estimators': 2000,
          'max_depth': 30,
          'min_samples_split': 15,
          'learning_rate': 0.01,
          'loss': 'ls'}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

pred3 = reg.predict(countTestVectDF)

pred3 = pd.DataFrame(pred3)
pred3.shape


pred3.to_excel('Testing output.xlsx')

# xgBoostModel = XGBRegressor()

In [112]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)

In [110]:
score

array([-0.83494389,  0.50470597,  0.20510294,  0.01761748, -0.22407102,
        0.80208468, -7.07507207,  0.51441545, -0.03473138,  0.47486173])

In [114]:
pred4 = pd.DataFrame(pred4)
pred4.shape


pred4.to_excel('Testing output.xlsx')